#📌 Extracción

##Bibliotecas a Utiliziar##

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

##Carga de datos JSON##

###Realizar carga de datos###

In [117]:
#Importar bibliotecas para realizar carga de datos desde Api
import requests
import json
'''
Cargar información
1. Traer la información del enlace
2. Generar archivo json
3. Convertir archivo json a dataframe
'''

data_telcom=requests.get('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json')
data=json.loads(data_telcom.text)
df=pd.DataFrame(data)
df.head()
#Se observa que cuenta con 4 columnas a normalizar

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


###Normalizar columnas###

In [118]:
# Copia del df original
'''
(Se aplica df.copy(), si no ambos apuntarán al mismo espacio de memoria)
'''
df_original = df.copy()

# Columnas a normalizar
columnas = ['customer', 'phone', 'internet', 'account']


#Normalización de cada columna de archivo JSON
'''
1. Crear una lista vacía para almacenar las columnas normalizadas
2. Recorrer cada columna con estructura JSON mediante for
3. Normalizar cada columna y almacenarla temporalmente en 'temp'
4. (Opcional) Renombrar columnas con el nombre de origen + campo
5. Agregar 'temp' a la lista columnas_normalizadas
6. Eliminar las columnas originales del DataFrame y guardar en df_base
7. Unir df_base con las columnas normalizadas para formar df_final
8. Utilizar costumerID como index
'''
columnas_normalizadas = []
for col in columnas:
    temp = pd.json_normalize(df_original[col])
    #temp.columns = [f'{col}_{subcol}' for subcol in temp.columns]
    columnas_normalizadas.append(temp)

# Mantener solo columnas no normalizadas
df_base = df_original.drop(columns=columnas)

# Unir todo: columnas base + columnas normalizadas
df_final = pd.concat([df_base] + columnas_normalizadas, axis=1)

#Cambio de index
df_final.set_index('customerID',inplace=True)
df_final.head()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
customerID,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformación

##Revisión de data frame##

In [119]:
#Extensión
df_final.shape
print(f'El dataframe tiene {df_final.shape[0]} filas y {df_final.shape[1]} columnas')

El dataframe tiene 7267 filas y 20 columnas


In [120]:
#Validación de nulos
df_final.isnull().sum()

,0
Churn,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [121]:
#Muestra de datos

df_final.sample(5)

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
customerID,,,,,,,,,,,,,,,,,,,,
8165-CBKXO,No,Male,0,Yes,Yes,66,No,No phone service,DSL,No,Yes,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),56.75,3708.4
0224-RLWWD,No,Female,1,No,No,2,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,44.30,89.3
9896-UYMIE,No,Male,0,No,No,66,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),114.30,7383.7
6024-RUGGH,No,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),25.20,1798.9
8739-QOTTN,No,Female,0,Yes,No,2,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.35,41.85


###Tipos de datos###

In [122]:
#Información de columnas
df_final.info()

'''
De acuerdo al diccionario de datos el tipo de cada columna debe ser:
customerID: object
Churn: boolean true/false
gender: object (puede ser categórica nominal)
SeniorCitizen: boolean true/false
Partner: boolean true/false
Dependents: boolean true/false
tenure: int64
PhoneService: boolean true/false
MultipleLines: boolean true/false
InternetService: object (puede ser categórica nominal)
OnlineSecurity: boolena true/false
OnlineBackup: boolena true/false
DeviceProtection: boolena true/false
TechSupport: boolena true/false
StreamingTV: boolena true/false
StreamingMovies: boolena true/false
Contract: object (puede ser categórica nominal)
PaperlessBilling: boolena true/false
PaymentMethod: object (puede ser categórica nominal)
Charges.Monthly: float64
Charges.Total: float64
'''

<class 'pandas.core.frame.DataFrame'>
Index: 7267 entries, 0002-ORFBO to 9995-HOTOH
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7267 non-null   object 
 1   gender            7267 non-null   object 
 2   SeniorCitizen     7267 non-null   int64  
 3   Partner           7267 non-null   object 
 4   Dependents        7267 non-null   object 
 5   tenure            7267 non-null   int64  
 6   PhoneService      7267 non-null   object 
 7   MultipleLines     7267 non-null   object 
 8   InternetService   7267 non-null   object 
 9   OnlineSecurity    7267 non-null   object 
 10  OnlineBackup      7267 non-null   object 
 11  DeviceProtection  7267 non-null   object 
 12  TechSupport       7267 non-null   object 
 13  StreamingTV       7267 non-null   object 
 14  StreamingMovies   7267 non-null   object 
 15  Contract          7267 non-null   object 
 16  PaperlessBilling  7267 non-null 

'\nDe acuerdo al diccionario de datos el tipo de cada columna debe ser:\ncustomerID: object\nChurn: boolean true/false\ngender: object (puede ser categórica nominal)\nSeniorCitizen: boolean true/false\nPartner: boolean true/false\nDependents: boolean true/false\ntenure: int64\nPhoneService: boolean true/false\nMultipleLines: boolean true/false\nInternetService: object (puede ser categórica nominal)\nOnlineSecurity: boolena true/false\nOnlineBackup: boolena true/false\nDeviceProtection: boolena true/false\nTechSupport: boolena true/false\nStreamingTV: boolena true/false\nStreamingMovies: boolena true/false\nContract: object (puede ser categórica nominal)\nPaperlessBilling: boolena true/false\nPaymentMethod: object (puede ser categórica nominal)\nCharges.Monthly: float64\nCharges.Total: float64\n'

In [128]:
#Cambio de tipo variable a columnas numéricas
'''
ternure y Charges.Monthly cuentan con el tipo correcto
Se debe eliminar espacios en blanco de Charges.Total y convertir
'''
#Validar Si presenta valores vacios
#df_final[df_final['Charges.Total'] == ''].count()

validacion=df_final.query('`Charges.Total` == " "').shape[0]

print(f"La columna Charges.Total presenta {validacion} valores vacíos de tipo \" \".")


La columna Charges.Total presenta 11 valores vacíos de tipo " ".


In [127]:
#Validación de datos vaciós en otras celdas
for col in df_final.columns:
    vacíos = df_final[col].astype(str).str.strip().eq("").sum()
    if vacíos > 0:
        print(f'La columna {col} presenta {vacíos} valores vacíos de tipo "".')



La columna Churn presenta 224 valores vacíos de tipo "".
La columna Charges.Total presenta 11 valores vacíos de tipo "".


In [138]:
#Los valores con espacios en blanco y ningun valor se deben eliminar.
#Eliminar vacios de churn
nan_churn=df_final[df_final['Churn'].astype(str).str.strip()==""].index
df_final.drop(nan_churn,inplace=True)
#Eliminar vacios de charge.total
nan_churn=df_final[df_final['Charges.Total'].astype(str).str.strip()==""].index
df_final.drop(nan_churn,inplace=True)

In [140]:
#Validar vacios
#Validación de datos vaciós en otras celdas
for col in df_final.columns:
    vacíos = df_final[col].astype(str).str.strip().eq("").sum()
    if vacíos > 0:
        print(f'La columna {col} presenta {vacíos} valores vacíos de tipo "".')



In [141]:
#Tranformar datos numéricos

df_final['Charges.Total']=df_final['Charges.Total'].apply(np.float64)
df_final.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0002-ORFBO to 9995-HOTOH
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null 

#📊 Carga y análisis

#📄Informe final